# Doppler Transform
This notebook illustrates the Doppler functional transform. This wideband Doppler effect operates over the entire sampled band by rescaling the input time series for a constant relative velocity and propagation speed.

In [ ]:
import numpy as np
from scipy.signal import find_peaks

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.constants import c

from torchsig.signals.signal_types import Signal
import torchsig.transforms.functional as F
from torchsig.utils.defaults import default_dataset
import torchsig.utils.dsp as dsp

Function for generating the input constant tone test signal.

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband Tone Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["tone"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

Place the input tone signal frequency at 0.2, then apply the wideband Doppler effect with a time scaling factor of 1.03. Estimate the frequency shift and compare it with the true shift.

In [ ]:
rng = np.random.default_rng(42)

fc = 0.2  # tone center frequency
N = 10000  # tone signal samples
sampling_rate = 4.0

# input tone signal
tone_bb_data = generate_tone_signal(num_iq_samples=N, scale=1.0).data
tone_data = tone_bb_data * np.exp(2j * np.pi * fc * np.arange(N) / sampling_rate)

# Doppler parameters
velocity = -1e7
propagation_speed = 2.9979e8
alpha = propagation_speed / (propagation_speed - velocity)

# apply Doppler transform
data_d = F.doppler(
    data=tone_data,
    velocity=velocity,
    propagation_speed=propagation_speed,
)
print(f"Doppler alpha factor: {alpha:.4f}")
print(f"True tone frequency pre-Doppler: {fc:.4f}")
print(f"True tone frequency post-Doppler: {fc*alpha:.4f}")

# estimate frequency shift
D = np.abs(np.fft.fft(data_d, norm="ortho"))
freqs = np.fft.fftfreq(len(D)) * sampling_rate
peaks, _ = find_peaks(D, height=0.1, distance=100)
print(f"Estimated tone frequency post-Doppler: {freqs[peaks[0]]:.4f}")

Plot the input and output spectrum magnitudes. Note that the output magnitude frequency response is not ideally clean - it has some residual frequency spurious components from the multiphase polyphase resampler that implements the wideband Doppler shift.

In [ ]:
plt.style.use("dark_background")
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(
    tone_data, Fs=sampling_rate, scale="linear", sides="twosided", color="white"
)
spectrum, freqs, _ = ax.magnitude_spectrum(
    data_d, Fs=sampling_rate, scale="linear", sides="twosided", color="red"
)
ax.set_yscale("log")
ax.set_ylabel("Magnitude")
plt.ylim([1e-15, None]);